# Find Optimal PSF
===========================

- creation 06/07/2016
- author Sylvie Dagoret-Campagne



Find HD163466 with Y3

In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from astropy.modeling import models
from astropy import units as u
from astropy import nddata
from astropy.io import fits

from astropy.table import Table
from astropy.table import Column

import ccdproc
print 'ccdproc version',ccdproc.__version__

from astropy.modeling import models

ccdproc version 1.0.1


In [8]:
import photutils
from astropy.stats import sigma_clipped_stats
from photutils import daofind
from photutils import CircularAperture
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize

In [9]:
from scipy import stats 
from scipy import ndimage
import os
from datetime import datetime, timedelta

In [10]:
from photutils.background import Background2D

In [11]:
import libMonocamBaseImages           # my tool library written to do that CCD reduction

In [12]:
now=datetime.utcnow()  # choose UTC time
datestr=str(now)
print 'standard date format for the analysis :',datestr
#  want the following format '2016-05-10T11:55:27.267'
date_of_analysis=now.strftime('%Y-%m-%dT%H:%M:%S')
print 'fits date format for the analysis : ',date_of_analysis

standard date format for the analysis : 2016-07-06 20:21:10.936277
fits date format for the analysis :  2016-07-06T20:21:10


## Definitions and Constants

In [13]:
object_name='HD163466_y3'

### input files

In [14]:
path='./HD163466_y3'
rootfilename='AssScImHD163466_y3_' 


NumStart=1
NumStop=10

### output file (table)

In [15]:
outputtablefile='HD163466_y3_1-10_TablePSF.fits'

### make the filelist

In [16]:
filelist=libMonocamBaseImages.BuildFilelist(path,rootfilename,start=NumStart,stop=NumStop,nbchar=1)

In [17]:
filelist

['./HD163466_y3/AssScImHD163466_y3_1.fits',
 './HD163466_y3/AssScImHD163466_y3_2.fits',
 './HD163466_y3/AssScImHD163466_y3_3.fits',
 './HD163466_y3/AssScImHD163466_y3_4.fits',
 './HD163466_y3/AssScImHD163466_y3_5.fits',
 './HD163466_y3/AssScImHD163466_y3_6.fits',
 './HD163466_y3/AssScImHD163466_y3_7.fits',
 './HD163466_y3/AssScImHD163466_y3_8.fits',
 './HD163466_y3/AssScImHD163466_y3_9.fits',
 './HD163466_y3/AssScImHD163466_y3_10.fits']

## Read Input files

In [18]:
allchannelallsciimages = []  # list of 16 lists of images series 
exposures_list = []        # sequential list of the exposures of the sky flats 
header_list = []           # list of headers
data_list = []
time_list = []                # date and time
basefile_list = []         # basefilename
dateobs_list = [] 
# get the primary block headers:
for image_file in filelist: 
    print image_file
    hdu_list = fits.open(image_file)
    basefile_list.append(os.path.basename(image_file))
    header=hdu_list[0].header
    exposure=header['EXPOSURE']
    exposures_list.append(exposure)
    dateobs_list.append(header['DATE-OBS'])
    header_list.append(header)
    data=ccdproc.CCDData.read(image_file, hdu=0,unit='adu') 
    data_list.append(data)

./HD163466_y3/AssScImHD163466_y3_1.fits
./HD163466_y3/AssScImHD163466_y3_2.fits
./HD163466_y3/AssScImHD163466_y3_3.fits
./HD163466_y3/AssScImHD163466_y3_4.fits
./HD163466_y3/AssScImHD163466_y3_5.fits
./HD163466_y3/AssScImHD163466_y3_6.fits
./HD163466_y3/AssScImHD163466_y3_7.fits
./HD163466_y3/AssScImHD163466_y3_8.fits
./HD163466_y3/AssScImHD163466_y3_9.fits
./HD163466_y3/AssScImHD163466_y3_10.fits


In [19]:
#basefile_list

## For control

uncomment for control

In [20]:
index=0

In [21]:
#print exposures_list[index]

In [22]:
#header_list[index]

In [23]:
#plt.imshow(data_list[index])

In [24]:
#bkg= Background2D(data_list[index], (100, 100), filter_size=(3, 3),method='median')

In [25]:
#data_list[index].data-bkg.background

## Background subtraction

In [26]:
correctedimage_list = []

In [27]:
for data in data_list:
    bkg= Background2D(data, (100, 100), filter_size=(3, 3),method='median')
    newimage=data-bkg.background
    correctedimage_list.append(newimage)

## Calculation of PSF

In [28]:
NBIMAGES=len(correctedimage_list)

In [29]:
def weighted_avg_and_std(values, weights):
    """
    Return the weighted average and standard deviation.

    values, weights -- Numpy ndarrays with the same shape.
    """
    average = np.average(values, weights=weights)
    variance = np.average((values-average)**2, weights=weights)  # Fast and numerically precise
    return (average, np.sqrt(variance))

In [30]:
image_psf=np.zeros((NBIMAGES,6))

In [31]:
DELTA_NBINSX=10
DELTA_NBINSY=10

In [32]:
# loop on images
index=-1
for image in correctedimage_list:
    index=index+1
    mean, median, std = sigma_clipped_stats(image, sigma=10.0, iters=5) 
    sources = daofind(image - median, fwhm=3.0, threshold=50.*std)
    selected_stars=sources.as_array()
    NBSTARS=selected_stars.shape[0]
    print 'image {} ==> NBSTARS = {}'.format(index,NBSTARS)
    image_psf[index,0]=index
    image_psf[index,1]=NBSTARS
    star_psfx=np.zeros(NBSTARS)
    star_psfy=np.zeros(NBSTARS)
    # loop on stars
    for istar in range(NBSTARS):
        X = int(selected_stars[istar][1])
        Y = int(selected_stars[istar][2])
        prf_image = image[Y-DELTA_NBINSY:Y+DELTA_NBINSY,X-DELTA_NBINSX:X+DELTA_NBINSX]
        oneprfX=prf_image.sum(axis=0)
        oneprfY=prf_image.sum(axis=1)
        if oneprfX.sum() == 0 or oneprfY.sum() == 0:
            star_psfx[istar]=0
            star_psfy[istar]=0
        else:
            posX,sigX=weighted_avg_and_std(np.arange(oneprfX.shape[0]),oneprfX)
            posY,sigY=weighted_avg_and_std(np.arange(oneprfY.shape[0]),oneprfY)
            star_psfx[istar]=sigX
            star_psfy[istar]=sigY
        
    all_sigx=star_psfx[np.logical_not(np.isnan(star_psfx))]
    all_sigy=star_psfy[np.logical_not(np.isnan(star_psfy))]
    all_sigx=all_sigx[all_sigx>2.4]
    all_sigy=all_sigy[all_sigy>2.4]
    print 'average prf(x) = {:2.2f} +/- {:2.2f} pixels ==> psf  {:2.2f} +/- {:2.2f} arcsec '.format(np.median(all_sigx),all_sigx.std(),np.median(all_sigx)*0.4*2.36,all_sigx.std()*0.4*2.26)
    print 'average prf(y) = {:2.2f} +/- {:2.2f} pixels ==> psf  {:2.2f} +/- {:2.2f} arcsec '.format(np.median(all_sigy),all_sigy.std(),np.median(all_sigy)*0.4*2.36,all_sigy.std()*0.4*2.26)
    image_psf[index,2]=np.median(all_sigx)
    image_psf[index,3]=np.median(all_sigy)
    image_psf[index,4]=all_sigx.std()
    image_psf[index,5]=all_sigy.std()

image 0 ==> NBSTARS = 7
average prf(x) = 3.14 +/- 0.06 pixels ==> psf  2.97 +/- 0.06 arcsec 
average prf(y) = 3.41 +/- 0.04 pixels ==> psf  3.22 +/- 0.04 arcsec 
image 1 ==> NBSTARS = 23
average prf(x) = 3.24 +/- 0.21 pixels ==> psf  3.05 +/- 0.19 arcsec 
average prf(y) = 3.67 +/- 0.31 pixels ==> psf  3.47 +/- 0.28 arcsec 
image 2 ==> NBSTARS = 30
average prf(x) = 3.28 +/- 0.15 pixels ==> psf  3.10 +/- 0.14 arcsec 
average prf(y) = 3.45 +/- 0.11 pixels ==> psf  3.25 +/- 0.10 arcsec 
image 3 ==> NBSTARS = 29
average prf(x) = 3.08 +/- 0.07 pixels ==> psf  2.91 +/- 0.07 arcsec 
average prf(y) = 3.40 +/- 0.18 pixels ==> psf  3.21 +/- 0.17 arcsec 


/Users/dagoret-campagnesylvie/anaconda/envs/pyastrophys/lib/python2.7/site-packages/ipykernel/__main__.py:9: RuntimeWarning: invalid value encountered in sqrt


image 4 ==> NBSTARS = 9
average prf(x) = 5.01 +/- 0.43 pixels ==> psf  4.73 +/- 0.39 arcsec 
average prf(y) = 5.13 +/- 0.93 pixels ==> psf  4.84 +/- 0.84 arcsec 
image 5 ==> NBSTARS = 4
average prf(x) = 3.67 +/- 0.13 pixels ==> psf  3.46 +/- 0.11 arcsec 
average prf(y) = 3.71 +/- 0.06 pixels ==> psf  3.51 +/- 0.05 arcsec 
image 6 ==> NBSTARS = 15
average prf(x) = 2.62 +/- 0.05 pixels ==> psf  2.47 +/- 0.05 arcsec 
average prf(y) = 2.76 +/- 0.05 pixels ==> psf  2.60 +/- 0.04 arcsec 
image 7 ==> NBSTARS = 12
average prf(x) = 3.03 +/- 0.09 pixels ==> psf  2.86 +/- 0.08 arcsec 
average prf(y) = 2.97 +/- 0.16 pixels ==> psf  2.81 +/- 0.15 arcsec 
image 8 ==> NBSTARS = 15
average prf(x) = 2.85 +/- 0.08 pixels ==> psf  2.69 +/- 0.07 arcsec 
average prf(y) = 2.90 +/- 0.04 pixels ==> psf  2.74 +/- 0.03 arcsec 
image 9 ==> NBSTARS = 20
average prf(x) = 2.75 +/- 0.10 pixels ==> psf  2.59 +/- 0.09 arcsec 
average prf(y) = 2.86 +/- 0.07 pixels ==> psf  2.70 +/- 0.06 arcsec 


## Create astropy table

In [33]:
t=Table(rows=image_psf,names=('num','nbstars','prfx','pfry','sig_prfx','sig_prfy'),dtype=('i4','i4','f8','f8','f8','f8'))

In [34]:
t

num,nbstars,prfx,pfry,sig_prfx,sig_prfy
int32,int32,float64,float64,float64,float64
0,7,3.14462939981,3.41466663301,0.0624903435569,0.0399877500438
1,23,3.23590764548,3.67168661807,0.210812621801,0.313203469327
2,30,3.28439425917,3.44808452428,0.153086049741,0.109240066938
3,29,3.07852576115,3.39865028239,0.0743265241544,0.182716405574
4,9,5.01067679435,5.12760526598,0.431963018433,0.931127413941
5,4,3.66881377389,3.71438578859,0.125757386795,0.0576096104765
6,15,2.62049631174,2.75934903687,0.053654463766,0.0477390482732
7,12,3.02620244552,2.97225160347,0.0890701171525,0.163930894688
8,15,2.85053312944,2.90077455,0.076245483463,0.038651577062


In [35]:
expo = Column(exposures_list, name='exposure')
file = Column(basefile_list, name='file')
time = Column(dateobs_list,name='time')

In [36]:
t.add_column(expo, index=1)
t.add_column(time, index=1)
t.add_column(file, index=1)

In [37]:
t

num,file,time,exposure,nbstars,prfx,pfry,sig_prfx,sig_prfy
int32,str26,str23,float64,int32,float64,float64,float64,float64
0,AssScImHD163466_y3_1.fits,2016-05-10T08:31:46.221,1.0,7,3.14462939981,3.41466663301,0.0624903435569,0.0399877500438
1,AssScImHD163466_y3_2.fits,2016-05-10T08:33:17.773,5.0,23,3.23590764548,3.67168661807,0.210812621801,0.313203469327
2,AssScImHD163466_y3_3.fits,2016-05-10T08:33:27.174,5.0,30,3.28439425917,3.44808452428,0.153086049741,0.109240066938
3,AssScImHD163466_y3_4.fits,2016-05-10T08:33:36.166,5.0,29,3.07852576115,3.39865028239,0.0743265241544,0.182716405574
4,AssScImHD163466_y3_5.fits,2016-05-10T08:38:53.776,3.0,9,5.01067679435,5.12760526598,0.431963018433,0.931127413941
5,AssScImHD163466_y3_6.fits,2016-05-10T08:39:35.943,3.0,4,3.66881377389,3.71438578859,0.125757386795,0.0576096104765
6,AssScImHD163466_y3_7.fits,2016-05-10T08:40:17.637,3.0,15,2.62049631174,2.75934903687,0.053654463766,0.0477390482732
7,AssScImHD163466_y3_8.fits,2016-05-10T08:41:40.854,3.0,12,3.02620244552,2.97225160347,0.0890701171525,0.163930894688
8,AssScImHD163466_y3_9.fits,2016-05-10T08:41:47.875,3.0,15,2.85053312944,2.90077455,0.076245483463,0.038651577062


## Write output file

In [38]:
t.write(outputtablefile,format='fits')

## Pandas

In [39]:
df=t.to_pandas()

In [40]:
df.describe()

,num,exposure,nbstars,prfx,pfry,sig_prfx,sig_prfy
count,10.00000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,4.50000,3.400000,16.400000,3.266613,3.427067,0.137860,0.195532
std,3.02765,1.264911,8.946756,0.681054,0.689866,0.113786,0.272582
min,0.00000,1.000000,4.000000,2.620496,2.759349,0.053654,0.038652
25%,2.25000,3.000000,9.750000,2.894450,2.918644,0.074806,0.050207
50%,4.50000,3.000000,15.000000,3.111578,3.406658,0.095132,0.090178
75%,6.75000,4.500000,22.250000,3.272273,3.615786,0.146254,0.178020
max,9.00000,5.000000,30.000000,5.010677,5.127605,0.431963,0.931127
